In [136]:
import os
import sys
import typing

from collections import namedtuple

import numpy as np
import pandas as pd

from nvdlib.nvd import NVD

In [25]:
sys.path.append('../src/')

In [6]:
from toolkit.preprocessing import NVDFeedPreprocessor

In [4]:
feed = NVD.from_recent()
# feed.update()

data = list(feed.cves())

In [8]:
transformed = NVDFeedPreprocessor(attributes=['configurations']).fit_transform(data)

In [13]:
# Series.configurations contains List[ConfigurationNode]
transformed_filtered = list(filter(lambda e: any(e.configurations), transformed))

In [19]:
pd.DataFrame(transformed_filtered)

,user,project,repository,configurations
0,cloudfoundry,cloud_controller_ng,https://github.com/cloudfoundry/cloud_controll...,[<nvdlib.model.ConfigurationNode object at 0x7...
1,curl,curl,https://github.com/curl/curl,[<nvdlib.model.ConfigurationNode object at 0x7...
2,faizzaidi,GeniXCMS-Version-1.1,https://github.com/faizzaidi/GeniXCMS-Version-1.1,[<nvdlib.model.ConfigurationNode object at 0x7...
3,phpipam,phpipam,https://github.com/phpipam/phpipam,[<nvdlib.model.ConfigurationNode object at 0x7...
4,tomato42,tlslite-ng,https://github.com/tomato42/tlslite-ng,[<nvdlib.model.ConfigurationNode object at 0x7...
5,lightSAML,lightSAML,https://github.com/lightSAML/lightSAML,[<nvdlib.model.ConfigurationNode object at 0x7...
6,OpenMPT,openmpt,https://github.com/OpenMPT/openmpt,[<nvdlib.model.ConfigurationNode object at 0x7...
7,goodrain-apps,chanzhieps,https://github.com/goodrain-apps/chanzhieps,[<nvdlib.model.ConfigurationNode object at 0x7...
8,torvalds,linux,https://github.com/torvalds/linux,[<nvdlib.model.ConfigurationNode object at 0x7...
9,VerSprite,research,https://github.com/VerSprite/research,[<nvdlib.model.ConfigurationNode object at 0x7...


In [52]:
def get_index(obj, index):
    try:
        el = obj[index]
    except IndexError:
        el = None
        pass
    
    return el

cpe_base_list = list(
    filter(
        lambda e: e is not None,
        [get_index(transformed_filtered[i].configurations[0].cpe, 0)
        for i in range(len(transformed_filtered))]
    )
)

In [137]:
from itertools import compress

CPE_VERSION_ATTRIBUTE_LIST = [
    'versionStartExcluding',
    'versionStartIncluding',
    'versionEndIncluding',
    'versionEndExcluding',
]


VICTIM_VERSION_OPERATOR_LIST = ['>', '>=', '<=', '<']


class VersionNode(namedtuple('VersionNode', CPE_VERSION_ATTRIBUTE_LIST)):
    
    def __new__(cls, cpe):
        return super(VersionNode, cls).__new__(
            cls,
            **{attr: getattr(cpe, attr, None) for attr in CPE_VERSION_ATTRIBUTE_LIST}
        )
        
    def __lt__(self, other):
        raise NotImplementedError
    
    def __gt__(self, other):
        raise NotImplementedError
    
    def __eq__(self, other):
        raise NotImplementedError
    
    @property
    def victims_notation(self):
        return VersionNode.get_victims_notation(self)
    
    @staticmethod
    def get_victims_notation(other: typing.Union[VersionNode, tuple]):
        version_string =  ",".join([
            f"{op}{version}" for op, version in zip(VICTIM_VERSION_OPERATOR_LIST, other)
            if version is not None
        ])
        
        return version_string

In [138]:
VersionNode(cpe_base_list[5]),  VersionNode(cpe_base_list[5]).victims_notation

(VersionNode(versionStartExcluding=None, versionStartIncluding=None, versionEndIncluding=None, versionEndExcluding='0.3.8'),
 '<0.3.8')

Inteded Usage: CVEVersionExtractor(cve)